In [1]:
# future functions
from __future__ import print_function 

# core scipy and numpy
import numpy as np
import scipy as sp

# pandas 
import pandas as pd
from IPython.display import display
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 135

# encoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

# manifold for embedding analysis
from sklearn import manifold

# Cross validation 
from sklearn.cross_validation import KFold

# Linear models 
from sklearn import linear_model

# Forests
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

# SVR
from sklearn.svm import SVR

# KNN
from sklearn.neighbors import KNeighborsRegressor

# matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

# python helpers 
from collections import namedtuple
from copy import copy

# neural network
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Merge, Reshape
from keras.layers.embeddings import Embedding

# XGboost for gradient-boosted decision trees
import xgboost as xgb

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
loadfile = lambda x: pd.read_csv(x, index_col=0)

In [3]:
Files = namedtuple('Files',['train','test'])
RawData = namedtuple('RawData',['train','test'])
ProcessedData = namedtuple('ProcessedData',['train','test'])

#### Load data 

In [4]:
rawfiles = Files(train='Data/train.csv.gz',
            test='Data/test.csv.gz')
raw = RawData(train=loadfile(rawfiles.train),
              test=loadfile(rawfiles.test))

In [5]:
raw.train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


#### Outliers 

In [6]:
# outliers = [  30,   88,  197,  462,  495,  523,  557,  632,  691,  825,  874,
#              898,  968,  970, 1169, 1170, 1182, 1423, 1432, 1453 ] # from outlier analysis
# outlier_ids = raw.train.index.isin(outliers)
# processed = ProcessedData(train=raw.train.loc[~outlier_ids ,:],
processed = ProcessedData(train=raw.train,
                    test=raw.test)

#### Preprocess categorical data

In [7]:
# concatenate all data 
cols = [ c for c in processed.train.columns if c != 'SalePrice' ]
full = pd.concat((processed.train.loc[:, cols],
                     processed.test))

# store important objects
label_encoders = {}
categorical_cols = []
categorical_col_nums = []
continuous_cols = []
medians = {}

# create lookup for numeric categorical columns
numeric_categorical = [
    'MSSubClass',
    'YearBuilt',
    'YearRemodAdd',
    'MoSold',
    'YrSold',
    'GarageYrBlt',
    'OverallQual',
    'OverallCond',
    'MiscVal',
]

# encode data and store objects
i = 0

for col, col_data in full.iteritems():
    
    # handle categorical columns 
    if col_data.dtype == object or col in numeric_categorical:
        le = LabelEncoder()
        col_data = le.fit_transform(col_data.fillna('Unk'))
        categorical_cols.append(col)
        categorical_col_nums.append(full.columns.get_loc(col))
        label_encoders[col] = le
    
    # handle continuous columns
    else:
        # handle nulls with median
        m = np.median(col_data.dropna())
        medians[col] = m
        
        col_data.loc[np.isnan(col_data)] = m
        continuous_cols.append(col)
    
    # store objects
    full[col] = col_data

/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


#### Log transfoms

In [8]:
get_skew = lambda x: sp.stats.skewtest(x)[1]<0.05
for c in continuous_cols:
    if get_skew(processed.train[c]):
        processed.train.loc[:, c] = np.log1p(processed.train[c])
        processed.test.loc[:, c] = np.log1p(processed.test[c])

#### Setup standard scaler and one hot encoder

In [9]:
# standard scaler
continous_scaler = StandardScaler()
_ = continous_scaler.fit(full.loc[:, continuous_cols])

# create one hot encoder
ohe = OneHotEncoder(categorical_features=categorical_col_nums)
_ = ohe.fit(full.as_matrix())

#### Categorical field analysis

In [10]:
categorical_field_analysis = []
for c in categorical_cols:
    # get pct freq coverage of top value 
    v = full[c].fillna('Unk').value_counts()
    first_pct = v.iloc[0]*1.0/sum(v)

    # get count of distinct values
    distinct_vals = set(full[c].values)
    d_cnt = len(distinct_vals)

    # calculate logical freq  
    logical_pct = 1.0/d_cnt
        
    # append
    categorical_field_analysis.append((c, first_pct, logical_pct, d_cnt))

categorical_field_analysis = pd.DataFrame(categorical_field_analysis,
                                 columns=['Cat_Col',
                                          'First_Freq',
                                          'Logical_Freq',
                                          'Distinct_Val_Cnt',])

In [11]:
@np.vectorize
def choose_m(freq, n):
    for i in range(1,n+1):
        if freq>=(1-float(i)/n):
            x = i
            break
    return x

In [12]:
categorical_field_analysis['m'] =\
choose_m(categorical_field_analysis.First_Freq,\
categorical_field_analysis.Distinct_Val_Cnt)

In [13]:
display(categorical_field_analysis)

,Cat_Col,First_Freq,Logical_Freq,Distinct_Val_Cnt,m
0,MSSubClass,0.369647,0.062500,16,11
1,MSZoning,0.775951,0.166667,6,2
2,Street,0.995889,0.500000,2,1
3,Alley,0.932169,0.333333,3,1
4,LotShape,0.636862,0.250000,4,2
5,LandContour,0.898253,0.250000,4,1
6,Utilities,0.998972,0.333333,3,1
7,LotConfig,0.730730,0.200000,5,2
8,LandSlope,0.951696,0.333333,3,1
9,Neighborhood,0.151764,0.040000,25,22


### NN model with entity embedding

Choose reshape size for each categorical column. Push continuous columns through, as-is.

#### model topology

In [25]:
models = []

for i, vals in categorical_field_analysis.T.iteritems():
    
    # gather reshaping components
    m = vals.Distinct_Val_Cnt
    new_m = vals.m
    
    # special cases 
    if vals.Cat_Col == 'YearBuilt':
        new_m = 10
    elif vals.Cat_Col == 'GarageYrBlt':
        new_m = 10
    elif vals.Cat_Col == 'YearRemodAdd':
        new_m = 10
    
    # create embedding for each feature
    entity_model = Sequential()
    entity_model.add(Embedding(m, new_m, input_length=1))
    entity_model.add(Reshape(target_shape=(new_m,)))
    models.append(entity_model)

n,m = full[continuous_cols].shape
continuous_model = Sequential()
continuous_model.add(Dense(m, input_dim=m))
models.append(continuous_model)

emb_model = Sequential()
emb_model.add(Merge(models, mode='concat'))
emb_model.add(Dropout(0.1))
emb_model.add(Dense(1000, init='uniform'))
emb_model.add(Activation('relu'))
emb_model.add(Dense(1000, init='uniform'))
emb_model.add(Activation('relu'))
emb_model.add(Dense(1))
emb_model.add(Activation('sigmoid'))
emb_model.compile(loss='mean_squared_error', optimizer='adam')

#### Preprocessing functions

In [26]:
_y = np.log(processed.train.SalePrice+1)
max_y = np.max(_y)

def nn_preprocessing_X(X_dat):
    X_out = []
    
    # categorical columns
    for c in categorical_cols:
        d = X_dat.loc[:, c].fillna('Unk').as_matrix()
        dt = label_encoders[c].transform(d)
        X_out.append(dt)
    
    # continuous columns
    for c in continuous_cols:
        X_dat.loc[np.isnan(X_dat[c]), c] = medians[c]
    continuous_ss = continous_scaler.transform(X_dat.loc[:, continuous_cols].as_matrix())
    X_out.append(continuous_ss)
    
    return X_out

def preprocessing_X(X_dat):
    X_dat = X_dat.copy()
    
    # categorical columns
    for c in categorical_cols:
        d = X_dat.loc[:, c].fillna('Unk').as_matrix()
        dt = label_encoders[c].transform(d)
        X_dat.loc[:, c] = dt
    
    # continuous columns
    for c in continuous_cols:
        X_dat.loc[np.isnan(X_dat[c]), c] = medians[c]
    continuous_ss = continous_scaler.transform(X_dat.loc[:, continuous_cols].as_matrix())
    X_dat.loc[:, continuous_cols] = continuous_ss
    
    X_dat = ohe.transform(X_dat.as_matrix())
    
    return X_dat

def preprocessing_Y(y_dat):
    return np.log(y_dat.values+1)/max_y

### Modeling

In [27]:
processed.train.shape 

(1460, 80)

In [28]:
ntrain = processed.train.shape[0]
ntest = processed.test.shape[0]

# set up K Folds
NFOLDS = 4
kf = KFold(ntrain, 
           n_folds=NFOLDS, 
           shuffle=False, 
           random_state=2)

#### Wrapper functions

In [29]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params={}):
#         params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        xtr = preprocessing_X(x_train)
        ytr = preprocessing_Y(y_train).ravel()
        self.clf.fit(xtr, ytr)

    def predict(self, x):
        xte = preprocessing_X(x)
        return self.clf.predict(xte)


class XgbWrapper:
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        xtr = preprocessing_X(x_train)
        ytr = preprocessing_Y(y_train).ravel()
        dtrain = xgb.DMatrix(xtr, label=ytr)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        xte = preprocessing_X(x)
        return self.gbdt.predict(xgb.DMatrix(xte))


class NnWrapper:
    def __init__(self, model, emb=True, nb_epoch=16, batch_size=8):
        self.model = copy(model)
        self.nb_epoch = nb_epoch
        self.batch_size = batch_size
        self.emb = emb
    
    def train(self, x_train, y_train):
        if self.emb:
            xtr = nn_preprocessing_X(x_train)
        else:
            xtr = preprocessing_X(x_train).A
        ytr = preprocessing_Y(y_train).ravel()
        self.model.fit(xtr, ytr, 
                 nb_epoch = self.nb_epoch,
                 batch_size = self.batch_size,
                 verbose = 0)
    
    def predict(self, x):
        if self.emb:
            xte = nn_preprocessing_X(x)
        else:
            xte = preprocessing_X(x).A
        return self.model.predict(xte).ravel()


def get_oof(clf):
    '''
        via:
        https://www.kaggle.com/eliotbarr/house-prices-
        advanced-regression-techniques/stacking-starter/code
    '''
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = X_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = X_train.iloc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(X_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


get_rmse = lambda x,y: np.mean(np.sqrt(((x.ravel()*max_y) - np.log(y.ravel()+1))**2))

#### Train models

In [30]:
X_train = processed.train.loc[:, 'MSSubClass':'SaleCondition']
y_train = processed.train['SalePrice']
X_test = processed.test.loc[:,'MSSubClass':'SaleCondition']

# vanilla NN 
n, m = preprocessing_X(X_train).shape
vanilla_nn_model = Sequential()
vanilla_nn_model.add(Dense(1000, init='uniform', input_shape=(m,)))
vanilla_nn_model.add(Activation('relu'))
vanilla_nn_model.add(Dropout(0.3))
vanilla_nn_model.add(Dense(500, init='uniform'))
vanilla_nn_model.add(Activation('relu'))
vanilla_nn_model.add(Dense(1))
vanilla_nn_model.add(Activation('sigmoid'))
vanilla_nn_model.compile(loss='mean_squared_error', optimizer='adam')

# create model objects
NN = NnWrapper(emb_model, emb=True)
NN2 = NnWrapper(vanilla_nn_model, emb=False)
XG = XgbWrapper(params={
                    'seed': 0 ,
                    'colsample_bytree': 0.7 ,
                    'silent': 1 ,
                    'subsample': 0.7 ,
                    'learning_rate': 0.035 ,
                    'objective': 'reg:linear' ,
                    'max_depth': 4 ,
                    'num_parallel_tree': 1 ,
                    'min_child_weight': 1 ,
                    'eval_metric': 'rmse' ,
                    'nrounds': 500 ,
                })
LS = SklearnWrapper(clf=linear_model.Lasso, params={'alpha':0.0001})
RG = SklearnWrapper(clf=linear_model.Ridge, params={'alpha':10.0})
RG2 = SklearnWrapper(clf=linear_model.Ridge, params={'alpha':10})
RF = SklearnWrapper(clf=RandomForestRegressor, params={
                                                'n_jobs': 4,
                                                'n_estimators': 100,
                                                'max_features': 0.2,
                                                'max_depth': 12,
                                                'min_samples_leaf': 2,
                                            })
ET = SklearnWrapper(clf=ExtraTreesRegressor, params={
                                                'n_jobs': 4,
                                                'n_estimators': 100,
                                                'max_features': 0.5,
                                                'max_depth': 12,
                                                'min_samples_leaf': 2,
                                            })
SVRL = SklearnWrapper(clf=SVR, params={'kernel':'linear','C':0.001,'epsilon':0.001})
SVRB = SklearnWrapper(clf=SVR, params={'kernel':'rbf','C':100.0,'epsilon':0.001})
KNN = SklearnWrapper(clf=KNeighborsRegressor, params={'n_jobs':4,'n_neighbors':6})

# run models 
nn_oof_train, nn_oof_test = get_oof(NN)
# nn2_oof_train, nn2_oof_test = get_oof(NN2)
# xgb_oof_train, xgb_oof_test = get_oof(XG)
# ls_oof_train, ls_oof_test = get_oof(LS)
# rg_oof_train, rg_oof_test = get_oof(RG)
# rg2_oof_train, rg2_oof_test = get_oof(RG2)
# rf_oof_train, rf_oof_test = get_oof(RF)
# et_oof_train, et_oof_test = get_oof(ET)
# svrl_oof_train, svrl_oof_test = get_oof(SVRL)
# svrb_oof_train, svrb_oof_test = get_oof(SVRB)
# knn_oof_train, knn_oof_test = get_oof(KNN)

In [31]:
print('NN1:\t{:,.4f}'.format(get_rmse(nn_oof_train, y_train)))
print('NN2:\t{:,.4f}'.format(get_rmse(nn2_oof_train, y_train)))
print('XGB:\t{:,.4f}'.format(get_rmse(xgb_oof_train, y_train)))
print('LS:\t{:,.4f}'.format(get_rmse(ls_oof_train, y_train)))
print('RG:\t{:,.4f}'.format(get_rmse(rg_oof_train, y_train)))
print('RG2:\t{:,.4f}'.format(get_rmse(rg2_oof_train, y_train)))
print('RF:\t{:,.4f}'.format(get_rmse(rf_oof_train, y_train)))
print('ET:\t{:,.4f}'.format(get_rmse(et_oof_train, y_train)))
print('SVRL:\t{:,.4f}'.format(get_rmse(svrl_oof_train, y_train)))
print('SVRB:\t{:,.4f}'.format(get_rmse(svrb_oof_train, y_train)))
print('KNN:\t{:,.4f}'.format(get_rmse(knn_oof_train, y_train)))

NN1:	0.1186
NN2:	0.1221
XGB:	0.0859
LS:	0.1034
RG:	0.1046
RG2:	0.1046
RF:	0.0992
ET:	0.1045
SVRL:	0.1039
SVRB:	0.1174
KNN:	0.1330


#### Combine models

In [21]:
X_train = np.concatenate((
                            nn_oof_train, 
                            nn2_oof_train, 
                            xgb_oof_train, 
                            ls_oof_train, 
                            rg_oof_train,
                            rg2_oof_train,
                            rf_oof_train,
                            et_oof_train,
                            svrl_oof_train,
                            svrb_oof_train,
                            knn_oof_train,
                         ), axis=1)
X_test = np.concatenate((
                            nn_oof_test, 
                            nn2_oof_test,
                            xgb_oof_test, 
                            ls_oof_test, 
                            rg_oof_test,
                            rg2_oof_test,
                            rf_oof_test,
                            et_oof_test,
                            svrl_oof_test,
                            svrb_oof_test,
                            knn_oof_test,
                        ), axis=1)

dtrain = xgb.DMatrix(X_train, label=preprocessing_Y(y_train))
dtest = xgb.DMatrix(X_test)

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.8,
    'silent': 1,
    'subsample': 0.6,
    'learning_rate': 0.01,
    'objective': 'reg:linear',
    'max_depth': 1,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'eval_metric': 'rmse',
}

res = xgb.cv(xgb_params, 
             dtrain, 
             num_boost_round=1000, 
             nfold=4, 
             stratified=False,
             early_stopping_rounds=25, 
             verbose_eval=200, 
             show_stdv=True)
best_nrounds = res.shape[0] - 1
xgb_final_model = xgb.train(xgb_params, dtrain, best_nrounds)

[0]	train-rmse:0.385644+0.000554692	test-rmse:0.385639+0.0016786
[200]	train-rmse:0.053677+0.00013877	test-rmse:0.053735+0.00152134
[400]	train-rmse:0.011954+0.000221778	test-rmse:0.0124047+0.00143359
[600]	train-rmse:0.00824675+0.000245496	test-rmse:0.0091385+0.00112028
[800]	train-rmse:0.00765175+0.000236574	test-rmse:0.00876775+0.000966394


In [22]:
calc_exp = lambda x: np.exp(x*max_y)-1
print('Ensemble:\t{:,.4f}+{:,.4f}'.format(calc_exp(res.iloc[-1,0]),calc_exp(res.iloc[-1,1])))

Ensemble:	0.1252+0.0128


In [23]:
predictions = np.exp(xgb_final_model.predict(dtest)*max_y)-1

In [24]:
submission = pd.DataFrame(np.vstack((raw.test.index.astype(str).ravel(), 
                                     predictions.ravel())).T,
                          columns=['Id','SalePrice'])
submission.to_csv('submission.csv', index=False)